In [1]:
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications import VGG16
from keras.layers import Dense, Flatten, Dropout, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from keras.models import Model
from keras.models import Sequential
from keras.optimizers import Adam

2024-04-12 11:27:53.529599: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-12 11:27:53.532457: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-12 11:27:53.562985: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 11:27:54.196488: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


**MODELS**

In [2]:
def transfert_model(keep_weights = True, num_classes = 10):
  
  # Load pre-trained VGG16 model without top layers
  base_model = VGG16(weights='imagenet' if keep_weights else None, include_top=False, input_shape=(224, 224, 3) if keep_weights  else (39,174,1))

  # Freeze convolutional base
  for layer in base_model.layers:
      layer.trainable = False

  # Add new fully connected layers
  x = Flatten()(base_model.output)
  output = Dense(num_classes, activation='softmax')(x)  # num_classes is the number of classes in your dataset

  # Create the new model
  model = Model(inputs=base_model.input, outputs=output)
  return model


def base_model(num_classes=10, input_shape=None, dropout_ratio=None):
    
    model = Sequential()
    if input_shape is None:
        model.add(Input(shape=(None, None, 1)))
    else:
        model.add(Input(shape=input_shape))
    model.add(Conv2D(filters=16, kernel_size=(2, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 3)))
    model.add(Conv2D(filters=32, kernel_size=(2, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64, kernel_size=(2, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=128, kernel_size=(2, 4), activation='relu'))
    model.add(GlobalAveragePooling2D())
    if dropout_ratio is not None:
        model.add(Dropout(dropout_ratio))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def improved_model(num_classes=10, input_shape=None, dropout_ratio=0.25):
    model = Sequential()
    if input_shape is None:
        model.add(Input(shape=(None, None, 1)))
    else:
        model.add(Input(shape=input_shape))

    model.add(Conv2D(filters=32, kernel_size=(2, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64, kernel_size=(2, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=128, kernel_size=(2, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=256, kernel_size=(2, 4), activation='relu'))
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(dropout_ratio))
    model.add(Dense(num_classes, activation='softmax'))
    return model

**UTILS**

In [3]:
def get_data(csv_file_path, random_state = 1):
  df = pd.read_csv(csv_file_path)
  df['features'] = [np.asarray(np.load(feature_path))
                    for feature_path in tqdm(df[f'mfcc_features_path'])]
  num_classes = 10
  df['labels_categorical'] = df['classID'].apply(
      lambda x: np.eye(num_classes)[x])

  # Add one dimension for the channel
  X = np.array(df['features'].tolist())
  X = X.reshape(X.shape + (1,))
  y = np.array(df['labels_categorical'].tolist())

  # As there is unbalance for some classes I am going to stratify it so we have the same proportion in train/test
  X_train, X_test, y_train, y_test = train_test_split(X,
                                                      y,
                                                      test_size=0.30,
                                                      random_state=random_state,
                                                      stratify=y)
  # Create validation and test
  X_test, X_val, y_test, y_val = train_test_split(X_test,
                                                  y_test,
                                                  test_size=0.5,
                                                  random_state=random_state,
                                                  stratify=y_test)
  return X_train, X_test, X_val, y_train, y_test, y_val

def schedule(epoch, lr):
    if epoch % 10 == 0 and epoch != 0:
        lr = lr * 0.95
    return lr

def launch_training(model, X_train, y_train, X_val, y_val, lr=0.001, bs=256, epochs=100, patience=10, decay=1):
  model.summary()
  model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy',
                metrics=['accuracy'])

  early_stopping = EarlyStopping(
      monitor='val_accuracy', patience=patience, restore_best_weights=True)
  checkpointer = ModelCheckpoint(
      filepath='saved_models/best_fcn.keras', monitor='val_accuracy', verbose=1, save_best_only=True)
  callbacks=[checkpointer, early_stopping]
  if decay < 1:
    lr_scheduler = LearningRateScheduler(schedule)
    callbacks.append(lr_scheduler)
  # Train the model
  history = model.fit(X_train,
                      y_train,
                      epochs=epochs,
                      batch_size=bs,
                      validation_data=(X_val, y_val),
                      callbacks=callbacks
                      )
  return model, history

def get_eval(model, history, X_test, y_test):
  # Plot training and validation loss
  plt.plot(history.history['accuracy'], label='Training Accuracy')
  plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.title('Training and Validation Accuracy')
  plt.show()
  best_val_accuracy = round(max(history.history['val_accuracy']),3)
  print(f"Best Validation Accuracy: {best_val_accuracy}")
  y_pred_probs = model.predict(X_test)
  y_pred = np.round(y_pred_probs)
  accuracy = round(accuracy_score(y_test, y_pred), 3 )
  print(f"Test Accuracy:{accuracy}")

In [ ]:
# # Replicate the grayscale channel along the channel dimension to create a 3-channel image
# X_train = tf.tile(X_train, [1, 1, 1, 3])
# X_val = tf.tile(X_val, [1, 1, 1, 3])
# X_test = tf.tile(X_test, [1, 1, 1, 3])
# X_train.shape

In [ ]:
LEARNING_RATE = 0.001
BATCH_SIZE = 256
EPOCHS = 200
PATIENCE = 20
DROPOUT = 0.5
DECAY = 1
RS = 1

X_train, X_test, X_val, y_train, y_test, y_val = get_data("extracted.csv", random_state = RS)
model = base_model(input_shape=X_train.shape[1:], dropout_ratio=DROPOUT)
trained_model, history = launch_training(model, X_train, y_train, X_val, y_val, lr = LEARNING_RATE, bs = BATCH_SIZE, epochs = EPOCHS, patience = PATIENCE, decay=DECAY)

In [ ]:
get_eval(trained_model, history, X_test, y_test)

In [5]:
import optuna

study = optuna.create_study(
    storage="sqlite:///db.sqlite3",  # Specify the storage URL here.
    study_name="audio-recognize2",
    direction="maximize"
)

[I 2024-04-12 11:28:29,345] A new study created in RDB with name: audio-recognize2


In [6]:
def objective(trial):

    LEARNING_RATE = trial.suggest_loguniform('learning_rate', 5e-4, 0.05)
    BATCH_SIZE = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256, 512])
    EPOCHS = 150
    PATIENCE = EPOCHS / 10
    DROPOUT = trial.suggest_float('dropout_ratio', 0.05, 0.5)
    DECAY = 0.95
    RS = 1
    
    model = improved_model(input_shape=X_train.shape[1:], dropout_ratio=DROPOUT)
    trained_model, history = launch_training(model, X_train, y_train, X_val, y_val, lr = LEARNING_RATE, bs = BATCH_SIZE, epochs = EPOCHS, patience = PATIENCE, decay=DECAY)
    best_val_accuracy = round(max(history.history['val_accuracy']),3)
    return best_val_accuracy

In [ ]:
X_train, X_test, X_val, y_train, y_test, y_val = get_data("extracted.csv", random_state = 1)

study.optimize(objective, n_trials=50, n_jobs=-1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8732/8732 [00:16<00:00, 532.55it/s]
/tmp/ipykernel_23377/314982782.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  LEARNING_RATE = trial.suggest_loguniform('learning_rate', 5e-4, 0.05)
2024-04-12 11:29:21.377673: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-12 11:29:21.379230: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to downl

Model: "sequential"

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 38, 171, 32)    │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 19, 85, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 18, 82, 64)     │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 9, 41, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 38, 128)     │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 19, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 3, 16, 256)     │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 38, 171, 32)    │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 19, 85, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 18, 82, 64)     │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 9, 41, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 38, 128)     │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 19, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 3, 16, 256)     │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Total params: 347,370 (1.33 MB)

 Trainable params: 347,370 (1.33 MB)

Epoch 1/150


 Non-trainable params: 0 (0.00 B)

Epoch 1/150
 72/191 ━━━━━━━━━━━━━━━━━━━━ 1:27 733ms/step - accuracy: 0.2296 - loss: 2.2270
Epoch 1: val_accuracy improved from -inf to 0.23206, saving model to saved_models/best_fcn.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step - accuracy: 0.1351 - loss: 3.3555 - val_accuracy: 0.2321 - val_loss: 2.0655 - learning_rate: 0.0011
Epoch 2/150
60/96 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.1178 - loss: 9.69402.16
Epoch 1: val_accuracy improved from -inf to 0.11450, saving model to saved_models/best_fcn.keras
24/24 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step - accuracy: 0.1121 - loss: 15.5595 - val_accuracy: 0.1145 - val_loss: 2.2598 - learning_rate: 0.0124
Epoch 2/150
79/96 ━━━━━━━━━━━━━━━━━━━━ 14s 876ms/step - accuracy: 0.1213 - loss: 8.32283
Epoch 1: val_accuracy improved from -inf to 0.45344, saving model to saved_models/best_fcn.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.2125 - loss: 2.3882 - val_accuracy: 0.4534 - val_loss: 1.5561 - learning_rate: 8.7901e-04
Epoch 2/150
155/19

In [ ]:
!optuna-dashboard sqlite:///db.sqlite3

In [ ]:
!pip install optuna-dashboard